# T-A3 v6.1 層別回帰分析

ソフィア推奨の追加解析：
- noise別にβ₁, β₂, β₃の符号が安定するか
- λ別にβ₁, β₂, β₃の符号が安定するか

**目的**: 全体回帰の結果が特定条件に偏っていないことを確認

In [ ]:
# Cell 1: Setup and Load Data
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Load data
SAVE_DIR = '/content/drive/MyDrive/T-A3_v6.1/'
df = pd.read_csv(f'{SAVE_DIR}T-A3_v6.1_results_FINAL.csv')

print(f"Loaded {len(df)} records")
print(f"\nFactors:")
print(f"  noise: {sorted(df['noise'].unique())}")
print(f"  lambda: {sorted(df['lambda'].unique())}")
print(f"  rho: {sorted(df['rho'].unique())}")

In [ ]:
# Cell 2: Helper Function for Regression
def run_regression(data, label=""):
    """
    Run the standard regression and return key statistics.
    """
    if len(data) < 10:
        return None
    
    # Standardize
    data = data.copy()
    data['phi_z'] = (data['phi_js'] - data['phi_js'].mean()) / (data['phi_js'].std() + 1e-10)
    data['A_z'] = (data['A'] - data['A'].mean()) / (data['A'].std() + 1e-10)
    data['phi_A_z'] = data['phi_z'] * data['A_z']
    
    # Model 2: Acc ~ Φ + A + Φ×A
    X = sm.add_constant(data[['phi_z', 'A_z', 'phi_A_z']])
    model = sm.OLS(data['acc'], X).fit()
    
    return {
        'label': label,
        'n': len(data),
        'R2': model.rsquared,
        'beta1': model.params['phi_z'],
        'beta1_p': model.pvalues['phi_z'],
        'beta2': model.params['A_z'],
        'beta2_p': model.pvalues['A_z'],
        'beta3': model.params['phi_A_z'],
        'beta3_p': model.pvalues['phi_A_z'],
    }

print("✓ Helper function ready!")

In [ ]:
# Cell 3: Stratified Regression by NOISE
print("=" * 70)
print("STRATIFIED REGRESSION BY NOISE (Integrated only)")
print("=" * 70)

df_int = df[df['arch'] == 'integrated'].copy()

results_noise = []
for noise in sorted(df_int['noise'].unique()):
    subset = df_int[df_int['noise'] == noise]
    result = run_regression(subset, f"noise={noise:.1f}")
    if result:
        results_noise.append(result)

# Display
print(f"\n{'Condition':<12} {'n':>6} {'R²':>8} {'β₁(Φ)':>10} {'β₂(A)':>10} {'β₃(Φ×A)':>10}")
print("-" * 60)
for r in results_noise:
    b1_sig = '*' if r['beta1_p'] < 0.05 else ' '
    b2_sig = '*' if r['beta2_p'] < 0.05 else ' '
    b3_sig = '*' if r['beta3_p'] < 0.05 else ' '
    print(f"{r['label']:<12} {r['n']:>6} {r['R2']:>8.4f} {r['beta1']:>+9.4f}{b1_sig} {r['beta2']:>+9.4f}{b2_sig} {r['beta3']:>+9.4f}{b3_sig}")

# Summary
print("\n(* = p < 0.05)")
print("\nSign Consistency Check:")
b1_signs = [r['beta1'] > 0 for r in results_noise]
b2_signs = [r['beta2'] > 0 for r in results_noise]
b3_signs = [r['beta3'] < 0 for r in results_noise]
print(f"  β₁ > 0: {sum(b1_signs)}/{len(b1_signs)} conditions")
print(f"  β₂ > 0: {sum(b2_signs)}/{len(b2_signs)} conditions")
print(f"  β₃ < 0: {sum(b3_signs)}/{len(b3_signs)} conditions")

In [ ]:
# Cell 4: Stratified Regression by LAMBDA
print("\n" + "=" * 70)
print("STRATIFIED REGRESSION BY LAMBDA (Integrated only)")
print("=" * 70)

results_lambda = []
for lam in sorted(df_int['lambda'].unique()):
    subset = df_int[df_int['lambda'] == lam]
    result = run_regression(subset, f"λ={lam:.1f}")
    if result:
        results_lambda.append(result)

# Display
print(f"\n{'Condition':<12} {'n':>6} {'R²':>8} {'β₁(Φ)':>10} {'β₂(A)':>10} {'β₃(Φ×A)':>10}")
print("-" * 60)
for r in results_lambda:
    b1_sig = '*' if r['beta1_p'] < 0.05 else ' '
    b2_sig = '*' if r['beta2_p'] < 0.05 else ' '
    b3_sig = '*' if r['beta3_p'] < 0.05 else ' '
    print(f"{r['label']:<12} {r['n']:>6} {r['R2']:>8.4f} {r['beta1']:>+9.4f}{b1_sig} {r['beta2']:>+9.4f}{b2_sig} {r['beta3']:>+9.4f}{b3_sig}")

# Summary
print("\n(* = p < 0.05)")
print("\nSign Consistency Check:")
b1_signs = [r['beta1'] > 0 for r in results_lambda]
b2_signs = [r['beta2'] > 0 for r in results_lambda]
b3_signs = [r['beta3'] < 0 for r in results_lambda]
print(f"  β₁ > 0: {sum(b1_signs)}/{len(b1_signs)} conditions")
print(f"  β₂ > 0: {sum(b2_signs)}/{len(b2_signs)} conditions")
print(f"  β₃ < 0: {sum(b3_signs)}/{len(b3_signs)} conditions")

In [ ]:
# Cell 5: Stratified Regression by NOISE × LAMBDA
print("\n" + "=" * 70)
print("STRATIFIED REGRESSION BY NOISE × LAMBDA (Integrated only)")
print("=" * 70)

results_full = []
for noise in sorted(df_int['noise'].unique()):
    for lam in sorted(df_int['lambda'].unique()):
        subset = df_int[(df_int['noise'] == noise) & (df_int['lambda'] == lam)]
        result = run_regression(subset, f"n={noise:.1f},λ={lam:.1f}")
        if result:
            result['noise'] = noise
            result['lambda'] = lam
            results_full.append(result)

# Display
print(f"\n{'Condition':<16} {'n':>5} {'R²':>7} {'β₁(Φ)':>9} {'β₂(A)':>9} {'β₃(Φ×A)':>9}")
print("-" * 60)
for r in results_full:
    b1_sig = '*' if r['beta1_p'] < 0.05 else ' '
    b2_sig = '*' if r['beta2_p'] < 0.05 else ' '
    b3_sig = '*' if r['beta3_p'] < 0.05 else ' '
    print(f"{r['label']:<16} {r['n']:>5} {r['R2']:>7.3f} {r['beta1']:>+8.3f}{b1_sig} {r['beta2']:>+8.3f}{b2_sig} {r['beta3']:>+8.3f}{b3_sig}")

# Summary
print("\n(* = p < 0.05)")
print("\nSign Consistency Check (all 18 conditions):")
b1_pos = sum(1 for r in results_full if r['beta1'] > 0)
b2_pos = sum(1 for r in results_full if r['beta2'] > 0)
b3_neg = sum(1 for r in results_full if r['beta3'] < 0)
print(f"  β₁ > 0: {b1_pos}/{len(results_full)} conditions")
print(f"  β₂ > 0: {b2_pos}/{len(results_full)} conditions")
print(f"  β₃ < 0: {b3_neg}/{len(results_full)} conditions")

In [ ]:
# Cell 6: Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Collect data for plotting
noise_levels = [r['noise'] for r in results_full]
lambda_levels = [r['lambda'] for r in results_full]
beta1_vals = [r['beta1'] for r in results_full]
beta2_vals = [r['beta2'] for r in results_full]
beta3_vals = [r['beta3'] for r in results_full]

# Panel A: β₁ by condition
ax = axes[0]
colors = ['#2ECC71' if b > 0 else '#E74C3C' for b in beta1_vals]
ax.bar(range(len(beta1_vals)), beta1_vals, color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_ylabel('β₁(Φ)', fontsize=12)
ax.set_xlabel('Condition (noise × λ)', fontsize=10)
ax.set_title('(A) β₁(Φ) by Condition', fontsize=12)
ax.set_xticks([])

# Panel B: β₂ by condition
ax = axes[1]
colors = ['#2ECC71' if b > 0 else '#E74C3C' for b in beta2_vals]
ax.bar(range(len(beta2_vals)), beta2_vals, color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_ylabel('β₂(A)', fontsize=12)
ax.set_xlabel('Condition (noise × λ)', fontsize=10)
ax.set_title('(B) β₂(A) by Condition', fontsize=12)
ax.set_xticks([])

# Panel C: β₃ by condition
ax = axes[2]
colors = ['#2ECC71' if b < 0 else '#E74C3C' for b in beta3_vals]  # Note: negative is expected
ax.bar(range(len(beta3_vals)), beta3_vals, color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_ylabel('β₃(Φ×A)', fontsize=12)
ax.set_xlabel('Condition (noise × λ)', fontsize=10)
ax.set_title('(C) β₃(Φ×A) by Condition', fontsize=12)
ax.set_xticks([])

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}T-A3_v6.1_stratified_betas.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"\n✓ Figure saved to {SAVE_DIR}")

In [ ]:
# Cell 7: Summary Table for Paper
print("\n" + "=" * 70)
print("SUMMARY TABLE FOR PAPER")
print("=" * 70)

# Create summary DataFrame
summary_data = []

# Overall
overall = run_regression(df_int, "Overall")
summary_data.append(overall)

# By noise
for r in results_noise:
    summary_data.append(r)

# By lambda
for r in results_lambda:
    summary_data.append(r)

# Create table
print("\n" + "=" * 80)
print(f"{'Stratum':<15} {'n':>6} {'R²':>7} {'β₁(Φ)':>10} {'β₂(A)':>10} {'β₃(Φ×A)':>10}")
print("=" * 80)

for r in summary_data:
    b1_str = f"{r['beta1']:+.4f}" + ('*' if r['beta1_p'] < 0.001 else '')
    b2_str = f"{r['beta2']:+.4f}" + ('*' if r['beta2_p'] < 0.001 else '')
    b3_str = f"{r['beta3']:+.4f}" + ('*' if r['beta3_p'] < 0.001 else '')
    print(f"{r['label']:<15} {r['n']:>6} {r['R2']:>7.4f} {b1_str:>10} {b2_str:>10} {b3_str:>10}")

print("=" * 80)
print("(* = p < 0.001)")

In [ ]:
# Cell 8: Final Verdict
print("\n" + "=" * 70)
print("FINAL VERDICT: SIGN CONSISTENCY")
print("=" * 70)

# Check all conditions
all_results = results_noise + results_lambda

b1_consistent = all(r['beta1'] > 0 for r in all_results)
b2_consistent = all(r['beta2'] > 0 for r in all_results)
b3_consistent = all(r['beta3'] < 0 for r in all_results)

print(f"\nβ₁(Φ) > 0 in ALL strata: {'✓ YES' if b1_consistent else '✗ NO'}")
print(f"β₂(A) > 0 in ALL strata: {'✓ YES' if b2_consistent else '✗ NO'}")
print(f"β₃(Φ×A) < 0 in ALL strata: {'✓ YES' if b3_consistent else '✗ NO'}")

if b1_consistent and b2_consistent and b3_consistent:
    print("\n" + "=" * 50)
    print("✓✓ ALL SIGNS CONSISTENT ACROSS STRATA")
    print("   Paper claims are robust!")
    print("=" * 50)
else:
    print("\n△ Some inconsistencies found - check details above")

# Save results to CSV
df_results = pd.DataFrame(summary_data)
df_results.to_csv(f'{SAVE_DIR}T-A3_v6.1_stratified_regression.csv', index=False)
print(f"\n✓ Results saved to {SAVE_DIR}T-A3_v6.1_stratified_regression.csv")